In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
count_2017 = pd.read_csv('../input/health-data/count_2017.csv')
count_2018 = pd.read_csv('../input/health-data/count_2018.csv')
count_2019 = pd.read_csv('../input/health-data/count_2019.csv')
count_2020 = pd.read_csv('../input/health-data/count_2020.csv')
to_predict = pd.read_csv('../input/health-data/to_predict.csv')
submit = pd.read_csv('../input/health-data/submit.csv')
count_df = pd.concat([count_2017,count_2018,count_2019,count_2020],axis=0)
import  os
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import mean_squared_error 
import time, datetime
import matplotlib.pyplot as plt
from pandas import Series,DataFrame
import seaborn as sns
test=to_predict
def time_process(df):
    df.date=pd.to_datetime(df.date)
    df['year']=df.date.dt.year     
    df['month']=df.date.dt.month
    df['day']=df.date.dt.day
    df['weekofyear']=df['date'].apply(lambda x: x.weekofyear)
    df['week_day']=df['date'].dt.weekday
    df=pd.get_dummies(df,columns=['week_day'])
    return df
count_df['date'] = count_df['date'].apply(str).apply(lambda x:datetime.datetime.strptime(x,'%Y%m%d')).astype('datetime64[D]')
test['date']=test['date'].apply(str).apply(lambda x:datetime.datetime.strptime(x,'%Y/%m/%d')).astype('datetime64[D]')
count_df = time_process(count_df)
test = time_process(to_predict)

In [ ]:
p1 = pd.read_csv('../input/fix-health-data/p1.csv')
p2 = pd.read_csv('../input/fix-health-data/p2.csv')
p3 = pd.read_csv('../input/fix-health-data/p3.csv')
p4 = pd.read_csv('../input/fix-health-data/p4.csv')
p5 = pd.read_csv('../input/fix-health-data/p5.csv')
p6 = pd.read_csv('../input/fix-health-data/p6.csv')

In [ ]:
c18=count_2018['count'][count_2018['admin_illness_name']=='上呼吸道感染']
c19=p1['count'][p1['admin_illness_name']=='上呼吸道感染']

In [ ]:
c19_3=pd.DataFrame(np.zeros(shape=(14,1))).astype(int)

In [ ]:
ctest=pd.concat([c18,c19,c19_3])

In [ ]:
from keras.models import Model, load_model
from keras.layers import Input, Conv1D, Dense, Dropout, Lambda, Concatenate
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pandas as pd
import pathlib
import gc
from datetime import timedelta
import os


pred_steps = 14
#数据处理部分
def preprocessing():

    df = ctest.values

    val_pred_start = len(df) - pred_steps
    val_pred_end = len(df)

    train_pred_start = val_pred_start - pred_steps - 1
    train_pred_end = val_pred_start - 1
    enc_length = train_pred_start

    train_enc_start = 0
    train_enc_end = train_enc_start + enc_length - 1

    val_enc_start = train_enc_start + pred_steps
    val_enc_end = val_enc_start + enc_length - 1

    print('Train encoding:', train_enc_start, '-', train_enc_end)
    print('Train prediction:', train_pred_start, '-', train_pred_end, '\n')
    print('Val encoding:', val_enc_start, '-', val_enc_end)
    print('Val prediction:', val_pred_start, '-', val_pred_end)
    series_array = np.array(df)
    series_array = series_array.reshape(len(series_array))
    return train_enc_start, train_enc_end, train_pred_start, train_pred_end, val_enc_start, val_enc_end, val_pred_start, val_pred_end, series_array


In [ ]:
#根据起始日期来划分数据
def get_time_block_series(series_array, start_date, end_date):
    return series_array[start_date:end_date]

#数据归一化，返回归一化后的数据，平均值，方差
def transform_series_encode(data):
    data_mean = data.mean(axis=0).reshape(-1, 1)
    data_std = data.std(axis=0).reshape(-1,1)
    epsilon = 1e-6
    data = (data - data_mean)/(data_std+epsilon)
    data = data.reshape((data.shape[0], data.shape[1], 1))
    return data, data_mean, data_std

#数据反归一化
def untransform_series_decode(data, data_mean, data_std):
    # data = data.reshape(data.shape[0], data.shape[1])
    data_std = data_std[0][0]
    data_mean = data_mean[0][0]
    data = data*data_std + data_mean
    data = data.reshape(len(data))
    # np.clip(np.power(10., data) - 1.0, 0.0, None)
    return  data

#数据归一化
def transform_series_decode(data, data_mean, data_std):
    epsilon = 1e-6 # prevent numerical error in the case std = 0
    data = (data - data_mean)/(data_std+epsilon)
    data = data.reshape((data.shape[0], data.shape[1], 1))
    return data


In [ ]:
#数据预测
def predict_sequences(input_sequences, batch_size):
    history_sequences = input_sequences.copy()
    print(history_sequences.shape)
    pred_sequences = np.zeros((history_sequences.shape[0], pred_steps, 1))  # initialize output (pred_steps time steps)
    print(pred_sequences.shape)
    for i in range(pred_steps):
        # record next time step prediction (last time step of model output)
        last_step_pred = model.predict(history_sequences,batch_size)[:, -1, 0]
        print("last step prediction first 10 channels")
        print(last_step_pred[0:10])
        print(i,last_step_pred.shape)
        pred_sequences[:, i, 0] = last_step_pred

        # add the next time step prediction to the history sequence
        history_sequences = np.concatenate([history_sequences,
                                           last_step_pred.reshape(-1, 1, 1)], axis=1)

    return pred_sequences



In [ ]:
#数据预测和画图
def predict_and_plot(encoder_input_data, pre_target_data, encode_series_mean, encode_series_std,batch_size, enc_tail_len=50, decoder_target_data=1):
    encode_series = encoder_input_data
    pred_series = predict_sequences(encode_series, batch_size)

    encode_series = encode_series.reshape(-1, 1)
    pred_series = pred_series.reshape(-1, 1)
    encode_series = encode_series.reshape(len(encode_series))
    pred_series = pred_series.reshape(len(pred_series))
    pre_target_data = pre_target_data.reshape(len(pre_target_data))

    encode_series = untransform_series_decode(encode_series,encode_series_mean, encode_series_std)
    pred_series = untransform_series_decode(pred_series,encode_series_mean, encode_series_std)
    # pre_target_data = untransform_series_decode(pre_target_data,encode_series_mean, encode_series_std)
    print(encode_series[-100:])

    plt.figure(figsize=(10, 6))
    plt.plot(range(0,100), encode_series[-100:],label = 'Encoding Series')
    plt.plot(range(100, 100 + len(pred_series)), pred_series, color='red', label='predict')
    plt.plot(range(100, 100 + len(pre_target_data)), pre_target_data, color='orange', label='target')

    plt.title('Encoder Series Tail of Length %d, Target Series, and Predictions' % enc_tail_len)
    plt.legend()
    plt.show()

model_name = 'Wavenet'

# load existing model
# load_previous_models = True
# if os.path.exists('{}.h5'.format(model_name)):
#     print('Load Previous Models')
#     model = load_model(model_name+'.h5')


#根据数据处理，得到数据的划分
train_enc_start, train_enc_end, train_pred_start, train_pred_end, val_enc_start, val_enc_end, val_pred_start, val_pred_end, series_array = preprocessing()



In [ ]:
#### Build neural networks ####
if not os.path.exists('{}.h5'.format(model_name)):
    n_filters = 128
    filter_width = 5
    dilation_rates = [2**i for i in range(12)]
    history_seq = Input(shape=(None, 1))
    x = history_seq

    for dilation_rate in dilation_rates:
        x = Conv1D(filters = n_filters,
                   kernel_size=filter_width,
                   padding='causal',
                   dilation_rate=dilation_rate)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(.2)(x)
    x = Dense(64)(x)
    x = Dense(1)(x)
    def slice(x, seq_length):
        return x[:, -seq_length:, :]

    pred_seq_train = Lambda(slice, arguments={'seq_length':pred_steps})(x)

    model = Model(history_seq, pred_seq_train)

model.summary()

#### Train neural networks ####
batch_size = 2**10
epochs = 100
encoder_input_data = get_time_block_series(series_array,train_enc_start, train_enc_end)
encoder_input_data, encode_series_mean, encode_series_std = transform_series_encode(encoder_input_data)
decoder_target_data = get_time_block_series(series_array,train_pred_start, train_pred_end)
decoder_target_data = transform_series_decode(decoder_target_data, encode_series_mean, encode_series_std)
# we append a lagged history of the target series to the input data,
# so that we can train with teacher forcing
lagged_target_history = decoder_target_data[:, :-1, :1]
encoder_input_data = np.concatenate([encoder_input_data, lagged_target_history], axis=1)
model.compile(Adam(), loss='mean_absolute_error')
history = model.fit(encoder_input_data, decoder_target_data,
                    batch_size=batch_size,
                    epochs=epochs)
# # save the model
# model.save(model_name + '.h5')

plt.figure()
plt.plot(history.history['loss'])

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.show()

pre_input_data = get_time_block_series(series_array, val_enc_start, val_enc_end)
pre_input_data, encode_series_mean, encode_series_std = transform_series_encode(pre_input_data)
pre_target_data = get_time_block_series(series_array,val_pred_start, val_pred_end)

predict_and_plot(pre_input_data, pre_target_data,encode_series_mean, encode_series_std, 2**10,decoder_target_data=pre_target_data)